In [46]:
import streamlit as st
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 40)

In [47]:
PATH = os.path.dirname(os.getcwd())
DATA_PATH = f"{PATH}\\Datas"
MODEL_PATH = f"{PATH}\\Model"
PATH

'c:\\Users\\chain\\2nd year\\datasci\\project\\ScopusData2018-2023'

In [48]:
def readCSV(filepath):
    df = pd.read_csv(filepath)
    df.index = df["Unnamed: 0"].values
    df.drop(labels= "Unnamed: 0",axis = 1,inplace=True)
    return df
def getUnique(df:pd.DataFrame, col:str):
    return sorted(
                df[col]
                .str.split(", ")
                .explode()
                .dropna()
                .unique()
            )

In [49]:
baseData = readCSV(f"{DATA_PATH}/data.csv")

In [50]:
baseData

,Publication Name,Title,Cited by Count,Subject Area,Abstracts,Author Keywords,Publisher,Author,Language,Publishing Date,Affiliation Names,Affiliated Cities,Affiliated Countries,Source Title
201800000,"Radiology in Global Health: Strategies, Implem...",Public health and international epidemiology f...,1.0,Medicine (all),NaN,NaN,Springer International Publishing,"Pongpirul K., Lungren M.P.",eng,2018-12-31,"Stanford University School of Medicine, Chulal...","Stanford, Bangkok, Bangkok, Stanford, Palo Alt...","United States, Thailand, Thailand, United Stat...","Radiology in Global Health: Strategies, Implem..."
201800001,Progress in Electromagnetics Research Symposium,Flexible Printed Active Antenna for Digital Te...,1.0,"Electrical and Electronic Engineering, Electro...","© 2018 The Institute of Electronics, Informati...",NaN,Institute of Electrical and Electronics Engine...,"Pratumsiri T., Janpugdee P.",eng,2018-12-31,Johns Hopkins Bloomberg School of Public Health,Baltimore,United States,Progress in Electromagnetics Research Symposium
201800002,Chemical Engineering Science,Parametric study of hydrogen production via so...,21.0,"Chemistry (all), Chemical Engineering (all), I...",© 2018 Elsevier LtdComputational fluid dynamic...,"Circulating fluidized bed, Computational fluid...",Elsevier Ltd,"Phuakpunk K., Chalermsinsuwan B., Putivisutisa...",eng,2018-12-31,Johns Hopkins Bloomberg School of Public Health,Baltimore,United States,Chemical Engineering Science
201800003,Applied Surface Science,Superhydrophobic coating from fluoroalkylsilan...,37.0,"Chemistry (all), Condensed Matter Physics, Phy...",© 2018 Elsevier B.V. A superhydrophobic/supero...,"Encapsulation, Fluoroalkylsilane, Natural rubb...",Elsevier B.V.,"Saengkaew J., Le D., Samart C., Sawada H., Nis...",eng,2018-12-31,"Hirosaki University, Chulalongkorn University,...","Hirosaki, Bangkok, Pathum Thani, Nagoya, Nakho...","Japan, Thailand, Thailand, Japan, Thailand, Th...",Applied Surface Science
201800004,Analytica Chimica Acta,Electrochemical impedance-based DNA sensor usi...,68.0,"Analytical Chemistry, Biochemistry, Environmen...",© 2018 Elsevier B.V. A label-free electrochemi...,"acpcPNA, Electrochemical impedance spectroscop...",Elsevier B.V.,"Teengam P., Siangproh W., Tuantranont A., Vila...",eng,2018-12-31,"Chulalongkorn University, Thailand National El...","Bangkok, Pathum Thani, Fort Collins, Bangkok","Thailand, Thailand, United States, Thailand",Analytica Chimica Acta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202302885,Catalysis Today,Long-chain bio-olefins production via oxidativ...,3.0,"Catalysis, Chemistry (all)",© 2021 Elsevier B.V.Long-chain α-olefins (≥ C1...,"Long-chain olefins, Mesoporous KIT-6, Oleic ac...",Elsevier B.V.,"Le D., Chaidherasuwet N., Rueangthaweep A., Ku...",eng,2023-01-01,"Chulalongkorn University, Center of Excellence...","Bangkok, Bangkok","Thailand, Thailand",Catalysis Today
202302886,Critical Reviews in Analytical Chemistry,Recent Developments and Applications of Microf...,11.0,Analytical Chemistry,"© 2021 Taylor & Francis Group, LLC.Nowadays, f...","Biological hazards, chemical hazards, food con...",Taylor and Francis Ltd.,"Alahmad W., Varanusupakul P., Varanusupakul P.",eng,2023-01-01,Center of Excellence on Petrochemical and Mate...,Bangkok,Thailand,Critical Reviews in Analytical Chemistry
202302887,Compare,"Social justice, education and peacebuilding: c...",5.0,Education,© 2021 The Author(s). Published by Informa UK ...,"conflict, Education, peacebuilding, social jus...",Routledge,Pherali T.,eng,2023-01-01,"Chulalongkorn University, UCL Institute of Edu...","Bangkok, London","Thailand, United Kingdom",Compare
202302888,Journal of Applied Aquaculture,Effects of black soldier fly (Hermetia illucen...,6.0,"Ecology, Aquatic Science",© 2021 Taylor & Francis.The effects of replaci...,"Anabas testudineus, Black soldier fly, fish me...",Taylor and Francis Ltd.,"Mapanao R., Jiwyam W., Nithikulworawong N., We...",eng,2023-01-

Authors Dataset

In [51]:
df = baseData.copy()

# 1) Split authors into list and explode so each row = one (paper, author)
df_long = (
    df.assign(Author=df["Author"].str.split(", "))
      .explode("Author")
)

# (optional) clean extra spaces
df_long["Author"] = df_long["Author"].str.strip()


In [52]:
author_stats = (
    df_long.groupby("Author")
           .agg(
               n_papers=("Author", "size"),
               total_citations=("Cited by Count", "sum"),
               average_citations=("Cited by Count", "mean")
           )
           .reset_index()
)



In [53]:
top3_per_author = (
    df_long.sort_values("Cited by Count", ascending=False)
           .groupby("Author")
           .head(3)[["Author", "Title", "Cited by Count"]]
)

top3_str_per_author = (
    top3_per_author
    .groupby("Author")
    .apply(
        lambda g: " | ".join(
            f"{i+1}. {row['Title']} ({row['Cited by Count']} cites)"
            for i, (_, row) in enumerate(g.iterrows())
        )
    )
    .reset_index(name="Top 3 Papers")
)

C:\Users\chain\AppData\Local\Temp\ipykernel_34600\3013971400.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [54]:
author_stats = (
    df_long
    .groupby("Author")
    .agg(
        n_papers=("Author", "size"),
        total_citations=("Cited by Count", "sum"),
        average_citations=("Cited by Count", "mean")
    )
    .reset_index()
    .sort_values("total_citations", ascending=False)
)

author_stats.head()


,Author,n_papers,total_citations,average_citations
62339,Wang Z.,1293,47005.0,36.353442
5061,Bhattacharya S.,1371,46000.0,33.552152
29956,Lee J.,1198,41686.0,34.796327
28561,Kumar A.,1118,41364.0,36.998211
62282,Wang J.,1124,37446.0,33.314947


In [55]:
author_full_stats = (
    author_stats
    .merge(top3_str_per_author, on="Author", how="left")
    .sort_values("total_citations", ascending=False)
)
top3_str_per_author = (
    top3_per_author
    .groupby("Author")
    .apply(
        lambda g: "\n".join(
            f"{i+1}. {row['Title']} ({row['Cited by Count']} cites)"
            for i, (_, row) in enumerate(g.iterrows())
        )
    )
    .reset_index(name="Top 3 Papers")
)

C:\Users\chain\AppData\Local\Temp\ipykernel_34600\1199951792.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [56]:
"""author_stats.head()
author_stats.columns = ["Author" , "Published" , "Total Citation" , "Average Citation"]"""
author_full_stats.columns = ["Author", "Published" , "Total Citations" , "Average Citations" , "Top 3 Papers"]
author_full_stats

,Author,Published,Total Citations,Average Citations,Top 3 Papers
0,Wang Z.,1293,47005.0,36.353442,1. Guidelines for the use and interpretation o...
1,Bhattacharya S.,1371,46000.0,33.552152,1. Guidelines for the use and interpretation o...
2,Lee J.,1198,41686.0,34.796327,1. Combined measurements of Higgs boson coupli...
3,Kumar A.,1118,41364.0,36.998211,1. Guidelines for the use and interpretation o...
4,Wang J.,1124,37446.0,33.314947,1. Guidelines for the use and interpretation o...
...,...,...,...,...,...
66544,Chutima D.,1,0.0,0.000000,1. Neuropsychiatric manifestations and sleep d...
66543,Sawekngam W.,1,0.0,0.000000,1. Instructional model development to enhance ...
66542,Sawetrattanasatian O.,1,0.0,0.000000,1. Desirability of Imagined Futures: Human-Cen...
66591,Tepjan S.,2,0.0,0.000000,1. Prevalence and Factors Associated with Suic...


In [57]:
author_full_stats.to_csv(f"{DATA_PATH}/author_stats.csv")

In [58]:
df = baseData.copy()

# 1) Basic publisher stats: number of papers + total citations
publisher_stats = (
    df.groupby("Publisher")
      .agg(
          n_papers=("Publisher", "size"),
          total_citations=("Cited by Count", "sum"),
          average_citations=("Cited by Count", "mean")
      )
      .reset_index()
)

In [59]:
top3_per_publisher = (
    df.sort_values("Cited by Count", ascending=False)
      .groupby("Publisher")
      .head(3)[["Publisher", "Title", "Cited by Count"]]
)

# 3) Turn top 3 into a single string column
top3_str_per_publisher = (
    top3_per_publisher
    .groupby("Publisher")
    .apply(
        lambda g: "\n".join(
            f"{i+1}. {row['Title']} ({row['Cited by Count']} cites)"
            for i, (_, row) in enumerate(g.iterrows())
        )
    )
    .reset_index(name="Top 3 Papers")
)

C:\Users\chain\AppData\Local\Temp\ipykernel_34600\3983896458.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [60]:
publisher_full_stats = (
    publisher_stats
    .merge(top3_str_per_publisher, on="Publisher", how="left")
    .sort_values("total_citations", ascending=False)
)

# preview
publisher_full_stats.columns = ["Publisher" , "Published" , "Total Citations", "Average Citations" , "Top 3 Papers"]
publisher_full_stats

,Publisher,Published,Total Citations,Average Citations,Top 3 Papers
322,Elsevier Ltd,1640,24508.0,14.943902,1. Paper-based electrochemical biosensor for d...
314,Elsevier B.V.,1518,23141.0,15.244401,1. Acute kidney injury from sepsis: current co...
755,Nature Research,447,6742.0,15.082774,1. Emerging MDR-Pseudomonas aeruginosa in fish...
683,MDPI AG,379,6588.0,17.382586,1. Solubility of cyclodextrins and drug/cyclod...
936,Springer,500,5170.0,10.340000,1. Acute-on-chronic liver failure: consensus r...
...,...,...,...,...,...
14,ASEAN Neurological Association,2,0.0,0.000000,1. The role of calcitonin gene-related peptide...
10,ARSMB-KVBMG,1,0.0,0.000000,1. Correlation between shoulder radiographic p...
5,ALTEX Edition,2,0.0,0.000000,1. Multilayered fibroblasts constructed by acc...
20,AZTI-Tecnalia,1,0.0,0.000000,1. Volatile profiles and aroma-active componen...


In [61]:
publisher_full_stats.to_csv(f"{DATA_PATH}/publisher_stats.csv")

In [62]:
df = baseData.copy()

# 1) Split subject areas into list and explode so each row = one (paper, subject area)
# Change "; " to ", " if your separator is comma instead of semicolon
df_long = (
    df.assign(**{"Subject Area": df["Subject Area"].str.split(", ")})
      .explode("Subject Area")
)

# Clean whitespace
df_long["Subject Area"] = df_long["Subject Area"].str.strip()


In [63]:

# 2) Basic subject-area stats: number of papers + total citations
subject_stats = (
    df_long.groupby("Subject Area")
           .agg(
               n_papers=("Subject Area", "size"),
               total_citations=("Cited by Count", "sum"),
               average_citations=("Cited by Count", "mean")
           )
           .reset_index()
)

# 3) Top 3 papers per subject area (by "Cited by Count")
top3_per_subject = (
    df_long.sort_values("Cited by Count", ascending=False)
           .groupby("Subject Area")
           .head(3)[["Subject Area", "Title", "Cited by Count"]]
)


In [64]:

# 4) Turn top 3 into a single string column
# Format: "1. Title A (120 cites) | 2. Title B (90 cites) | 3. Title C (45 cites)"
top3_str_per_subject = (
    top3_per_subject
    .groupby("Subject Area")
    .apply(
        lambda g: "\n".join(
            f"{i+1}. {row['Title']} ({row['Cited by Count']} cites)"
            for i, (_, row) in enumerate(g.iterrows())
        )
    )
    .reset_index(name="Top 3 Papers")
)


C:\Users\chain\AppData\Local\Temp\ipykernel_34600\4072353924.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [65]:

# 5) Merge everything together
subject_full_stats = (
    subject_stats
    .merge(top3_str_per_subject, on="Subject Area", how="left")
    .sort_values("total_citations", ascending=False)
)

# preview
subject_full_stats.head()
subject_full_stats.columns = ["Subject Area", "Published", "Total Citations","Average Citations", "Top 3 Papers"]
subject_full_stats

,Subject Area,Published,Total Citations,Average Citations,Top 3 Papers
234,Nuclear and High Energy Physics,425,11756.0,27.661176,1. Search for invisible decays of a Higgs boso...
53,Chemistry (all),905,10735.0,11.861878,1. Understanding Degradation Mechanisms and Im...
293,Renewable Energy,652,10510.0,16.119632,1. Consensus statement for stability assessmen...
322,Sustainability and the Environment,652,10510.0,16.119632,1. Consensus statement for stability assessmen...
32,Biochemistry,1016,10374.0,10.210630,1. Implications of TP53 allelic state for geno...
...,...,...,...,...,...
283,Psychology (miscellaneous),5,1.0,0.200000,1. The Effect of Cognitive-Behavioral Group Th...
247,Optometry,1,0.0,0.000000,1. Macular pigment optical density in a Thai s...
113,Emergency Nursing,1,0.0,0.000000,1. Novel tool for assessing the quality of fee...
131,Family Practice,1,0.0,0.000000,1. Angiotensin-Converting Enzyme Inhibitor-Ind...


In [66]:
subject_full_stats.to_csv(f"{DATA_PATH}//subject_area_stats.csv")

In [67]:
subject_full_stats.columns

Index(['Subject Area', 'Published', 'Total Citations', 'Average Citations',
       'Top 3 Papers'],
      dtype='object')

In [68]:
df = baseData.copy()

# 1) Split and explode Subject Area
df["Subject Area"] = (
    df["Subject Area"]
      .fillna("")
      .str.split(",")
)

df = df.explode("Subject Area")
df["Subject Area"] = df["Subject Area"].str.strip()
df = df[df["Subject Area"] != ""]

# 2) Keep only subject areas that appear in subject_full_stats (optional but clean)


In [69]:
valid_subjects = set(subject_full_stats["Subject Area"])
df = df[df["Subject Area"].isin(valid_subjects)]

# 3) Parse date
df["Publishing Date"] = pd.to_datetime(df["Publishing Date"], errors="coerce")
df = df.dropna(subset=["Publishing Date"])

# (recommended) Aggregate by month instead of exact day to make charts smoother
df["date"] = df["Publishing Date"].dt.to_period("M").dt.to_timestamp()


In [70]:

# 4) Aggregate: papers & citations per (date, subject)
agg = (
    df.groupby(["date", "Subject Area"])
      .agg(
          papers=("Title", "count"),
          citations=("Cited by Count", "sum"),
      )
      .reset_index()
)

In [71]:

papers_wide = (
    agg
    .pivot(index="date", columns="Subject Area", values="papers")
    .sort_index()
)
citations_wide = (
    agg
    .pivot(index="date", columns="Subject Area", values="citations")
    .sort_index()
)


papers_wide.index.name = "date"
citations_wide.index.name = "date"

In [72]:
papers_wide.to_csv(f"{DATA_PATH}/line_chart_datesXpapers.csv")
citations_wide.to_csv(f"{DATA_PATH}/line_chart_datesXcitations.csv")

In [73]:
df = baseData[["Author"]].copy()
df["Author"] = df["Author"].str.split(", ")
df["from"] = df["Author"].apply(lambda x:x.pop(0))
df = df[["from", "Author"]]
df.columns = ["from","to"]
df = df.explode("to")

In [74]:

tmp = df[["from", "to"]].copy().dropna(axis=0,how="any")

# Canonical ordering of each pair (min, max) so (A,B) == (B,A)
endpoints = np.sort(tmp[["from", "to"]].to_numpy(), axis=1)
tmp["u"] = endpoints[:, 0]
tmp["v"] = endpoints[:, 1]

# Group by canonical endpoints and count occurrences
weighted = (
    tmp.groupby(["u", "v"], as_index=False)
       .size()
       .rename(columns={"u": "from", "v": "to", "size": "weight"})
)


In [75]:
weighted = weighted[weighted["weight"] > 100]
weighted.sort_values("weight", inplace=True)

weighted


,from,to,weight
46907,Cole J.E.,Tumasyan A.,101
101835,Paktinat Mehdiabadi S.,Sirunyan A.M.,101
88426,Li Y.Y.,Sirunyan A.M.,101
90231,Lopes De Sá R.,Sirunyan A.M.,101
111517,Ruiz Vargas J.C.,Sirunyan A.M.,101
...,...,...,...
114634,Sharma A.,Sirunyan A.M.,875
83592,Kumar A.,Sirunyan A.M.,913
86305,Lee J.,Sirunyan A.M.,1025
116690,Sirunyan A.M.,Wang Z.,1034


In [76]:
weighted.to_csv(f"{DATA_PATH}/authors_network.csv")


In [77]:
df = baseData[["Affiliated Countries"]].copy()
df["Affiliated Countries"] = df["Affiliated Countries"].str.split(", ")
df.dropna(axis=0 , how="any" , inplace=True)
df["from"] = df["Affiliated Countries"].apply(lambda x:x.pop(0))
df = df[["from", "Affiliated Countries"]]
df.columns = ["from","to"]
df = df.explode("to")
df

,from,to
201800000,United States,Thailand
201800000,United States,Thailand
201800000,United States,United States
201800000,United States,United States
201800000,United States,United States
...,...,...
202302885,Thailand,Thailand
202302886,Thailand,NaN
202302887,Thailand,United Kingdom
202302888,Thailand,Thailand


In [78]:

tmp = df[["from", "to"]].copy().dropna(axis=0,how="any")

# Canonical ordering of each pair (min, max) so (A,B) == (B,A)
endpoints = np.sort(tmp[["from", "to"]].to_numpy(), axis=1)
tmp["u"] = endpoints[:, 0]
tmp["v"] = endpoints[:, 1]

# Group by canonical endpoints and count occurrences
weighted = (
    tmp.groupby(["u", "v"], as_index=False)
       .size()
       .rename(columns={"u": "from", "v": "to", "size": "weight"})
)


In [79]:

weighted.sort_values("weight", inplace=True)

weighted

,from,to,weight
13,Albania,Pakistan,1
12,Albania,Mongolia,1
11,Albania,Italy,1
10,Albania,Israel,1
9,Albania,India,1
...,...,...,...
1241,Germany,India,6162
1290,Germany,Turkey,8198
2497,Thailand,Thailand,15746
1246,Germany,Italy,16403


In [80]:
weighted.to_csv(f"{DATA_PATH}/countries_network.csv")

In [81]:
df = baseData.copy()
df_country = df[["Affiliated Countries", "Cited by Count", "Title"]].copy()

# drop rows with no country info
df_country = df_country.dropna(subset=["Affiliated Countries"])

# split "A, B, C" -> ["A","B","C"]
df_country["Country"] = df_country["Affiliated Countries"].str.split(r",\s*")

# explode to long format
df_country = df_country.explode("Country")
df_country["Country"] = df_country["Country"].str.strip()

In [82]:
country_stats = (
    df_country
    .groupby("Country")
    .agg(
        n_papers=("Title", "nunique"),          # or "size" if one row = one paper
        total_citations=("Cited by Count", "sum"),
        average_citations=("Cited by Count", "mean")
    )
    .reset_index()
)


In [83]:

df_country = df[["Affiliated Countries", "Cited by Count", "Title"]].copy()

# drop rows with no country info
df_country = df_country.dropna(subset=["Affiliated Countries"])

# split "A, B, C" -> ["A","B","C"]
df_country["Country"] = df_country["Affiliated Countries"].str.split(r",\s*")

# explode to long format
df_country = df_country.explode("Country")
df_country["Country"] = df_country["Country"].str.strip()

In [84]:
# sort so the biggest citation papers come first within each country
tmp = df_country.sort_values(
    ["Country", "Cited by Count"],
    ascending=[True, False]
)

# take top 3 rows per country
top3 = tmp.groupby("Country").head(3)

# create a nice "Title (citations)" string
top3 = top3.assign(
    PaperInfo=lambda d: d["Title"] + " (" + d["Cited by Count"].astype(str) + ")"
)

# join the 3 strings per country
top3_str = (
    top3.groupby("Country")["PaperInfo"]
    .agg(" | ".join)
    .reset_index(name="Top 3 Papers")
)


In [85]:
country_summary = country_stats.merge(top3_str, on="Country", how="left")
country_summary.iloc[0,0] = "Cote Dlvoire"
country_summary.sort_values("Country" , inplace=True)


In [91]:
countryCoord = pd.read_csv(f"{PATH}\\longitude-latitude.csv")
countryCoord = countryCoord[["Country","Latitude","Longitude"]]

countryCoord

,Country,Latitude,Longitude
0,Aruba,12.521110,-69.968338
1,Afghanistan,33.939110,67.709953
2,Angola,-11.202692,17.873887
3,Anguilla,18.220554,-63.068615
4,Åland Islands,60.250716,20.374146
...,...,...,...
316,North Borneo,5.250000,117.000000
317,Northern Rhodesia,-13.133897,27.849332
318,Australasia,-41.000000,124.000000
319,Australasia,-41.000000,124.000000


In [87]:
df_merged = country_summary.merge(
    countryCoord[['Country', 'Latitude', 'Longitude']],
    on='Country',
    how='left'
)

# 2) Find countries that did NOT get matched (no latitude/longitude)
missing_countries = (
    df_merged.loc[df_merged['Latitude'].isna(), 'Country']
    .dropna()
    .unique()
    .tolist()
)

print("Missing countries:", missing_countries)

df_merged


Missing countries: ['Cote Dlvoire', 'Democratic Republic Congo', 'Federated States of Micronesia', 'None']


,Country,n_papers,total_citations,average_citations,Top 3 Papers,Latitude,Longitude
0,Afghanistan,4,207.0,51.750000,Global Retinoblastoma Presentation and Analysi...,33.939110,67.709953
1,Albania,6,236.0,39.333333,Global Retinoblastoma Presentation and Analysi...,41.153332,20.168331
2,Algeria,19,774.0,27.642857,Advancing the global public health agenda for ...,28.033886,1.659626
3,Angola,1,146.0,146.000000,Global Retinoblastoma Presentation and Analysi...,-11.202692,17.873887
4,Argentina,90,14402.0,96.657718,Guidelines for the use and interpretation of a...,-38.416097,-63.616672
...,...,...,...,...,...,...,...
167,Venezuela,6,476.0,68.000000,Practical management of acne for clinicians: A...,6.423750,-66.589730
168,Viet Nam,511,10121.0,12.342683,European Position Paper on Rhinosinusitis and ...,14.058324,108.277199
169,Yemen,8,303.0,33.666667,Global Retinoblastoma Presentation and Analysi...,15.552727,48.516388
170,Zambia,8,808.0,73.454545,Health equity and COVID-19: Global perspective...,-13.133897,27.849332


In [88]:
df_merged.loc[36 , ["Latitude", "Longitude"]] = [7.546855, -5.5471]
df_merged.loc[41 , ["Latitude", "Longitude"]] = [-4.038333, 21.758664]
df_merged.loc[50 , ["Latitude", "Longitude"]] = [7.425554, 150.550812]
df_merged = df_merged[df_merged["Country"]!= "None"]

In [89]:
df_merged.columns = ["Country" , "Published" , "Total Citations", "Average Citations" , "Top 3 Papers" , "Latitude" , "Longitude"]
df_merged

,Country,Published,Total Citations,Average Citations,Top 3 Papers,Latitude,Longitude
0,Afghanistan,4,207.0,51.750000,Global Retinoblastoma Presentation and Analysi...,33.939110,67.709953
1,Albania,6,236.0,39.333333,Global Retinoblastoma Presentation and Analysi...,41.153332,20.168331
2,Algeria,19,774.0,27.642857,Advancing the global public health agenda for ...,28.033886,1.659626
3,Angola,1,146.0,146.000000,Global Retinoblastoma Presentation and Analysi...,-11.202692,17.873887
4,Argentina,90,14402.0,96.657718,Guidelines for the use and interpretation of a...,-38.416097,-63.616672
...,...,...,...,...,...,...,...
167,Venezuela,6,476.0,68.000000,Practical management of acne for clinicians: A...,6.423750,-66.589730
168,Viet Nam,511,10121.0,12.342683,European Position Paper on Rhinosinusitis and ...,14.058324,108.277199
169,Yemen,8,303.0,33.666667,Global Retinoblastoma Presentation and Analysi...,15.552727,48.516388
170,Zambia,8,808.0,73.454545,Health equity and COVID-19: Global perspective...,-13.133897,27.849332


In [90]:
df_merged.to_csv(f"{DATA_PATH}\\mapData.csv")